In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from os import walk
from sklearn.cluster import KMeans


In [9]:
related_img1 = cv2.imread("satellite_imgs/related_img1.png")
related_img2 = cv2.imread("satellite_imgs/related_img2.png")
unrelated_img = cv2.imread("satellite_imgs/unrelated_img.png")

sfo1 = cv2.imread("satellite_imgs/sfo1.png")
sfo2 = cv2.imread("satellite_imgs/sfo2.png")
sfo1_rot = cv2.imread("satellite_imgs/sfo1_rotated.png")

In [10]:
def imshow(img):
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.show()

In [11]:
class VecKMEncoder:
    def __init__(self):
        self.d = 1024
        self.alpha = 0.1
        self.W = np.stack([self.strict_standard_normal(self.d) for _ in range(2)], axis=0) * self.alpha
        self.detector = cv2.SIFT_create(1000)
        
    def strict_standard_normal(self, d):
        y = np.linspace(0, 1, d+2)
        x = norm.ppf(y)[1:-1]
        np.random.shuffle(x)
        return x
    
    def encode(self, img):
        kp = self.detector.detect(img, None)
        pts = cv2.KeyPoint_convert(kp).astype(np.complex128)
        Z = np.exp(1j * pts @ self.W).sum(axis=0)
        Z = Z / np.linalg.norm(Z)
        return Z

    def similarity(self, x, y):
        return np.absolute(np.sum(x * y.conj()))

In [12]:
class DLCosineEncoder:
    def __init__(self):
        # Load the pretrained model
        self.model = models.resnet18(pretrained=True)
        # Use the model object to select the desired layer
        self.layer = self.model._modules.get('avgpool')

        # Set model to evaluation mode
        self.model.eval()
        self.scaler = transforms.Resize((224, 224))
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
        self.to_tensor = transforms.ToTensor()
        self.sift_function = cv2.SIFT_create(nfeatures=500)
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    def get_kpt_img(self, rgb_img):
        gray_img = cv2.cvtColor(rgb_img,cv2.COLOR_BGR2GRAY) # Converting the RGB image to grayscale
         # Creating an instance of the SIFT Function
        keypoints, features = self.sift_function.detectAndCompute(gray_img,None) # Computing the set of keypoints and features for the image
        blank = np.zeros(rgb_img.shape).astype(np.uint8)
        for i in range(len(keypoints)):
    #         print(int(keypoints[i].pt[1]),int(keypoints[i].pt[0]))
            cv2.circle(blank, (int(keypoints[i].pt[0]),int(keypoints[i].pt[1])), 3, (255,255,255), -1)
    #         blank[int(keypoints[i].pt[1]),int(keypoints[i].pt[0])] = [255,255,255]
        return blank
    
    def get_vector(self, img):
        # 2. Create a PyTorch Variable with the transformed image
        t_img = Variable(self.normalize(self.scaler(self.to_tensor(img))).unsqueeze(0))
        # 3. Create a vector of zeros that will hold our feature vector
        #    The 'avgpool' layer has an output size of 512
        my_embedding = torch.zeros(1,512,1,1)
        # 4. Define a function that will copy the output of a layer

        def copy_data(m, i, o):
            my_embedding.copy_(o.data)
        # 5. Attach that function to our selected layer
        h = self.layer.register_forward_hook(copy_data)
        # 6. Run the model on our transformed image
        self.model(t_img)
        # 7. Detach our copy function from the layer
        h.remove()
        # 8. Return the feature vector
        return my_embedding
    
    def similarity(self, x, y):
        return self.cos(x.unsqueeze(0),y.unsqueeze(0))


In [13]:
class HistEncoder():

    def __init__(self,siftD=128,num_clusters=64):
        self.siftD = siftD
        self.num_clusters = num_clusters
        self.clustering_model = self.create_hist_model()

    def detect_sift_features(self,rgb_img):
        gray_img = cv2.cvtColor(rgb_img,cv2.COLOR_BGR2GRAY)
        sift_function = cv2.SIFT_create(nfeatures=128)
        keypoints, features = sift_function.detectAndCompute(gray_img,None)
        return keypoints, features

    def create_hist_model(self):
        all_features = []
        filenames = next(walk("./satellite_imgs/"), (None, None, []))[2]  # [] if no file
        for image_path in filenames:
            img = cv2.imread("./satellite_imgs/" + image_path)
            keypoints, features = self.detect_sift_features(img)
            all_features.append(features)
        features_array = np.concatenate(all_features,axis=0).reshape((-1,self.siftD))
        clustering_model = KMeans(n_clusters=64,n_init="auto")
        clustering_model.fit(features_array)
        return clustering_model

    def hist_encode(self,rgb_img):
        keypoints, features = self.detect_sift_features(rgb_img)
        features = np.array(features).reshape((-1,self.siftD))
        feature_labels = self.clustering_model.predict(features).reshape((-1))
        hist, bin_edges = np.histogram(feature_labels,bins=[i for i in range(self.num_clusters)])
        return np.array(hist).astype(np.float32)

    def hist_similarity_correlation(self,hist1,hist2):
        sim = cv2.compareHist(hist1,hist2,0)
        return sim

    def hist_similarity_intersection(self,hist1,hist2):
        sim = cv2.compareHist(hist1,hist2,2)
        return sim

In [14]:
DLCos = DLCosineEncoder()

vec1 = DLCos.get_vector(related_img1)
vec1_resize = vec1.view(512)
vec2 = DLCos.get_vector(related_img2)
vec2_resize = vec2.view(512)
vec3 = DLCos.get_vector(unrelated_img)
vec3_resize = vec3.view(512)
sfo_vec1 = DLCos.get_vector(sfo1)
sfo_vec1_resize = sfo_vec1.view(512)
sfo_vec2 = DLCos.get_vector(sfo2)
sfo_vec2_resize = sfo_vec2.view(512)
sfo_vec3 = DLCos.get_vector(sfo1_rot)
sfo_vec3_resize = sfo_vec3.view(512)

In [ ]:
cos_sim1_2 = DLCos.similarity(vec1_resize, vec2_resize)
cos_sim2_3 = DLCos.similarity(vec2_resize, vec3_resize)
cos_sim1_3 = DLCos.similarity(vec1_resize, vec3_resize)

sfo_cos_sim1_2 = DLCos.similarity(sfo_vec1_resize, sfo_vec2_resize)
sfo_cos_sim2_3 = DLCos.similarity(sfo_vec2_resize, sfo_vec3_resize)
sfo_cos_sim1_3 = DLCos.similarity(sfo_vec1_resize, sfo_vec3_resize)

cos_1_sfo = DLCos.similarity(vec1_resize, sfo_vec3_resize)

print(cos_sim1_2, cos_sim2_3, cos_sim1_3, sfo_cos_sim1_2, sfo_cos_sim2_3, sfo_cos_sim1_3, cos_1_sfo)

In [16]:
VecKM = VecKMEncoder()

vec1 = VecKM.encode(related_img1)
vec2 = VecKM.encode(related_img2)
vec3 = VecKM.encode(unrelated_img)

sfo_vec1 = VecKM.encode(sfo1)
sfo_vec2 = VecKM.encode(sfo2)
sfo_vec3 = VecKM.encode(sfo1_rot)

In [ ]:
sim1_2 = VecKM.similarity(vec1, vec2)
sim2_3 = VecKM.similarity(vec2, vec3)
sim1_3 = VecKM.similarity(vec1, vec3)

sfo_sim1_2 = VecKM.similarity(sfo_vec1, sfo_vec2)
sfo_sim2_3 = VecKM.similarity(sfo_vec2, sfo_vec3)
sfo_sim1_3 = VecKM.similarity(sfo_vec1, sfo_vec3)

one_sfo = VecKM.similarity(vec1, sfo_vec3)

print(sim1_2, sim2_3, sim1_3, sfo_sim1_2, sfo_sim2_3, sfo_sim1_3, one_sfo)

In [18]:
HistE = HistEncoder()

vec1 = HistE.hist_encode(related_img1)
vec2 = HistE.hist_encode(related_img2)
vec3 = HistE.hist_encode(unrelated_img)

sfo_vec1 = HistE.hist_encode(sfo1)
sfo_vec2 = HistE.hist_encode(sfo2)
sfo_vec3 = HistE.hist_encode(sfo1_rot)

In [ ]:
sim1_2 = HistE.hist_similarity_correlation(vec1, vec2)
sim2_3 = HistE.hist_similarity_correlation(vec2, vec3)
sim1_3 = HistE.hist_similarity_correlation(vec1, vec3)

sfo_sim1_2 = HistE.hist_similarity_correlation(sfo_vec1, sfo_vec2)
sfo_sim2_3 = HistE.hist_similarity_correlation(sfo_vec2, sfo_vec3)
sfo_sim1_3 = HistE.hist_similarity_correlation(sfo_vec1, sfo_vec3)

one_sfo = HistE.hist_similarity_correlation(vec1, sfo_vec3)

print(sim1_2, sim2_3, sim1_3, sfo_sim1_2, sfo_sim2_3, sfo_sim1_3, one_sfo)

In [ ]:
sim1_2 = HistE.hist_similarity_intersection(vec1, vec2)
sim2_3 = HistE.hist_similarity_intersection(vec2, vec3)
sim1_3 = HistE.hist_similarity_intersection(vec1, vec3)

sfo_sim1_2 = HistE.hist_similarity_intersection(sfo_vec1, sfo_vec2)
sfo_sim2_3 = HistE.hist_similarity_intersection(sfo_vec2, sfo_vec3)
sfo_sim1_3 = HistE.hist_similarity_intersection(sfo_vec1, sfo_vec3)

one_sfo = HistE.hist_similarity_intersection(vec1, sfo_vec3)

print(sim1_2, sim2_3, sim1_3, sfo_sim1_2, sfo_sim2_3, sfo_sim1_3, one_sfo)